In [8]:
%load_ext autoreload
%autoreload 2

In [4]:
from datasets import AsyncPaginatedFetcher

query = {  
    "queries": [
        {
            "type" : "queryText",
            "value": "temperature"
        }      
    ],
    "summary": False
}

# fetcher = AsyncPaginatedFetcher(
#     'https://data.noaa.gov/onestop/api/search/search/collection',
#     query,
#     500, 
#     results_per_page=50,
#     n_workers=5,
#     # result_handler=handler
# )

# await fetcher.fetch_all()

# await main('https://data.noaa.gov/onestop/api/search/search/collection', 80, 20, handler, query)

In [13]:
import httpx
headers = {'Accept': 'application/json'}

temperature = httpx.post('https://data.noaa.gov/onestop/api/search/search/collection', headers=headers, json=query).json()

In [14]:
temperature

{'data': [{'attributes': {'stagedDate': 1688888428983,
    'fileIdentifier': 'gov.noaa.ncei:SeaSurfaceTemperature',
    'title': '.Sea Surface Temperature',
    'description': 'Derived from processed satellite observations. These data are usually within 0.5 degrees Celsius uncertainty for ocean areas and 1 degree Celsius uncertainty for coastal areas.',
    'keywords': [],
    'spatialBounding': {'type': 'Polygon',
     'coordinates': [[[-97.9, 17.1],
       [-78.66, 17.1],
       [-78.66, 31.6],
       [-97.9, 31.6],
       [-97.9, 17.1]]]},
    'isGlobal': False,
    'dataFormats': [{'name': 'PNG'}],
    'dataFormat': ['PNG'],
    'links': [{'linkFunction': 'completeMetadata'}],
    'linkProtocol': [],
    'serviceLinkProtocol': [],
    'individualNames': [],
    'organizationNames': [],
    'thumbnail': 'Not provided.',
    'citeAsStatements': [],
    'crossReferences': [],
    'largerWorks': [],
    'legalConstraints': [],
    'dsmmAverage': 0.0,
    'serviceLinks': []},
   'id': '

In [12]:
import pandas as pd
temp_df = pd.json_normalize(temperature['data'])
# temp_df
# df = pd.json_normalize(data['data'])
temp_df['attributes.spatialBounding.coordinates'] = temp_df['attributes.spatialBounding.coordinates'].astype(str)temp_df['attributes.spatialBounding.coordinates'] = temp_df['attributes.spatialBounding.coordinates'].astype(str)
# temp_df['onestop_query'] = query['queries'][0]['value']

SyntaxError: invalid syntax (3612704382.py, line 5)

In [21]:
print(temp_df.columns)

Index(['id', 'type', 'attributes.stagedDate', 'attributes.fileIdentifier',
       'attributes.title', 'attributes.description', 'attributes.keywords',
       'attributes.spatialBounding.type',
       'attributes.spatialBounding.coordinates', 'attributes.isGlobal',
       'attributes.dataFormats', 'attributes.dataFormat', 'attributes.links',
       'attributes.linkProtocol', 'attributes.serviceLinkProtocol',
       'attributes.individualNames', 'attributes.organizationNames',
       'attributes.thumbnail', 'attributes.citeAsStatements',
       'attributes.crossReferences', 'attributes.largerWorks',
       'attributes.legalConstraints', 'attributes.dsmmAverage',
       'attributes.serviceLinks', 'attributes.beginDate',
       'attributes.beginYear', 'attributes.beginDayOfYear',
       'attributes.beginDayOfMonth', 'attributes.beginMonth',
       'attributes.endDate', 'attributes.endYear', 'attributes.endDayOfYear',
       'attributes.endDayOfMonth', 'attributes.endMonth',
       'attribu

In [15]:
import duckdb
import pandas as pd
# No need to import duckdb_engine
#  jupysql will auto-detect the driver needed based on the connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [16]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [17]:
%sql duckdb:///:default:

In [26]:
%%sql 

CREATE TABLE onestop_datasets AS SELECT * FROM read_json_auto('./data/onestop/*.json', format='array')

InvalidInputException: Invalid Input Error: JSON transform error in file "./data/onestop/1688932612.json", in record/value 1: Object {"name":"HDF","version":"4"} has unknown key "version"
Try increasing 'sample_size', reducing 'maximum_depth', specifying 'columns', 'format' or 'records' manually, setting 'ignore_errors' to true, or setting 'union_by_name' to true when reading multiple files with a different structure.

In [7]:
%sql DESCRIBE TABLE onestop_datasets

CatalogException: Catalog Error: Table with name onestop_datasets does not exist!
Did you mean "pg_database"?
LINE 1: DESCRIBE TABLE onestop_datasets
                       ^

In [46]:
%sql ALTER TABLE onestop_datasets ADD COLUMN onestop_query VARCHAR

,Success


In [49]:
from datasets import async_pool_fetcher

con = duckdb.connect(database=':default:')

def duckdb_conn_wrapper(duckdb_con, table):
    def handle_results(data):
        local_con = duckdb_con.cursor()
        df = pd.json_normalize(data['data'])
        df['onestop_query'] = query['queries'][0]
        local_con.execute(f"""
            INSERT INTO {table} SELECT * FROM df
        """)
    return handle_results

handler = duckdb_conn_wrapper(con, 'onestop_datasets')
